# Terminologia tensorial



*   ***Orden:*** Numero de indices que tiene.
*   ***Rango (o rango de descomposicion):*** Para tensores con orden $>$ 2, es la dimension minima especificada de la factorizacion de un tensor con respecto a alguna particion (agrupacion) de sus indices. 
*   ***Dimension:*** Tamaño del indice de un tensor, es decir, numero maximo de valores que un indice puede tener

<img src="https://static.wixstatic.com/media/d91e93_636996276a41489a8ffb2b534ae62836~mv2.png/v1/fill/w_273,h_153,al_c,q_90/d91e93_636996276a41489a8ffb2b534ae62836~mv2.webp" width="250">


Para nuestros propositos, los tensores seran simplemente arreglos multidimensionales de numeros reales o complejos


In [1]:
#Inicializar algunos tensores con numpy y tensornetwork

import numpy as np
import tensornetwork
import tensorflow as tf
net = tensornetwork.TensorNetwork() # Implementacion de una red tensorial

#Crear un tensor con entradas generadas aleatoriamente, de orden 3, dimensiones: 2,3,4

A = np.random.rand(2,3,4)
a = net.add_node(A)

In [2]:
#Veamos la forma del tensor A
print(A)

[[[0.6703595  0.53518907 0.23409105 0.61047003]
  [0.1816934  0.08035142 0.5727741  0.22497991]
  [0.03478381 0.38731988 0.36972085 0.6777268 ]]

 [[0.09105677 0.46112459 0.96702023 0.23869212]
  [0.57945049 0.92334889 0.29585892 0.60187019]
  [0.21533049 0.12765094 0.01045382 0.25517896]]]


In [3]:
# Así es como se accede al tensor subyacente.
print(a.tensor)

Tensor("Const:0", shape=(2, 3, 4), dtype=float64)


In [4]:
#Ahora, como en tensorflow, para ejecutar el codigo se hace a partir de una sesión

with tf.Session() as sess:
  at = sess.run(a.tensor)

print(at)

[[[0.6703595  0.53518907 0.23409105 0.61047003]
  [0.1816934  0.08035142 0.5727741  0.22497991]
  [0.03478381 0.38731988 0.36972085 0.6777268 ]]

 [[0.09105677 0.46112459 0.96702023 0.23869212]
  [0.57945049 0.92334889 0.29585892 0.60187019]
  [0.21533049 0.12765094 0.01045382 0.25517896]]]


# Notacion Diagramatica

*  Cada tensor será representado por un circulo conectado a lineas que corresponden a sus indices.

* Cada linea estará etiquetada con una letra necesaria para relacionar la ecuacion con el diagram.

![texto alternativo](https://static.wixstatic.com/media/d91e93_6a91aeea94304044a62cc32d064aaf61~mv2.png/v1/fill/w_660,h_340,al_c,q_90/d91e93_6a91aeea94304044a62cc32d064aaf61~mv2.webp)


In [5]:
# En TensorNetwork las lineas libres son creadas automaticamente con la creacion del nodo.
# podemos acceder a ellas de la siguiente manera.
edgei = a.get_edge(0) # a[0]
edgej = a.get_edge(1) # a[1]
edgek = a.get_edge(2) # a[2]

# Las dimensiones de cada linea son
print('Dimensiones de las lineas')
print(edgei.dimension)
print(edgej.dimension)
print(edgek.dimension)

print('Orden del tensor')
print(a.get_rank())

Dimensiones de las lineas
2
3
4
Orden del tensor
3


# Contraccion

Un indice compartido por dos tensores indica una contraccion (o sumatoria) sobre éste indice.

![texto alternativo](https://static.wixstatic.com/media/d91e93_4a2852f173884f0c96aa869aa6524ee7~mv2.png/v1/fill/w_954,h_325,al_c,q_90/d91e93_4a2852f173884f0c96aa869aa6524ee7~mv2.webp)

# Operaciones de permutar y remodelar (reshape)

* ***Permutar:*** cambia el orden del índice de un tensor (pero no cambia el número de índices)

* ***remodelar:*** combina una colección de índices en un índice más grande (o viceversa), puede cambiar el número de índices pero no la dimensión total.

![texto alternativo](https://static.wixstatic.com/media/d91e93_6afe662a2ff44b3c8df018f0ac787b09~mv2.png/v1/fill/w_950,h_368,al_c,q_90/D18.webp)

Nota:

*    Hay que permutar los nodos para afirmar que los ejes de su resultado están en el orden correcto

*    Contraer sin reordenar puede producir calculos intermedios, lo que se traduce en mas costo computacional

* Permutar reordena el almacenamiento de los elementos de un tensor en la memoria de la computadora, por lo que incurre en un costo computacional (a menudo no despreciable). Por el contrario, la función de remodelación deja los elementos de un tensor sin cambios en la memoria, solo cambia los metadatos de cómo debe interpretarse el tensor (y por lo tanto incurre en un costo insignificante).

In [6]:
#Permutar y remodelar en numpy

#Permutar el tensor A de orden 4 con dimensiones 1,2,3,4

A = np.random.rand(1,2,3,4)

Atilda = A.transpose(3,0,1,2)

print(Atilda.shape) #Debe imprimir (4,1,2,3)

#Remodelar el tensor B de orden 3 con dimensiones 4,4,4

B = np.random.rand(4,4,4)

Btilda = B.reshape(4,4*4)

print(Btilda.shape) #Debe imprimir (4,16)

(4, 1, 2, 3)
(4, 16)


In [7]:
#Permutar y remodelar en tensornetwork

#Permutar el tensor D de orden 3 con dimensiones 1,2,3

D = net.add_node(np.zeros((1, 2, 3)))
d1 = D[0]
d2 = D[1]
d3 = D[2]
D.reorder_edges([d3, d1, d2])

print(D.tensor.shape) # Debe imprimir (3, 1, 2)

#Remodelar el tensor E de orden 3 con dimensiones 2,3,4

E = net.add_node(np.ones((2, 3, 4)))
e1 = E[0]
e2 = E[1]
e3 = E[2]
flattened_edge = net.flatten_edges([E[0], E[1]])

E.reorder_edges([flattened_edge,e3])

print(E.tensor.shape) #Debe imprimir (6,4)

(3, 1, 2)
(6, 4)


# Contraccion de tensor binario

Permutar y remodelar permiten que una contracción entre un par de tensores (que llamamos una contracción de tensor binario) se pueda escribir como una multiplicación matricial. 

![texto alternativo](https://static.wixstatic.com/media/d91e93_a62d3c7786494eada0bb5a2f3518d662~mv2.png/v1/fill/w_864,h_195,al_c,q_90/d91e93_a62d3c7786494eada0bb5a2f3518d662~mv2.webp)

Metodologia:

1.   Permutar los tensores A y B tal que los indices a contraer esten contiguos entre ambos tensores
2.   Remodelar los tensores (agrupar entre indices a contraer e indices externos) a una matriz 
3.   Hacer la multiplicación de matrices
4.   Remodele la matriz C en un tensor, realice la permutación final si el orden del índice deseado es diferente del orden actual

![texto alternativo](https://static.wixstatic.com/media/d91e93_8c99c11e840e40e49bceaf9751df3158~mv2.png/v1/fill/w_555,h_490,al_c,q_90/d91e93_8c99c11e840e40e49bceaf9751df3158~mv2.webp)

In [8]:
##### Contraccion de tensor binario en numpy
i = 1; m = 2; j = 3; n = 4; k = 5; l = 6

#Cijkl = Aimjn*Bmkln

A = np.random.rand(i,m,j,n)  #Aimjn
B = np.random.rand(m,k,l,n)  #Bmkln

print('dimension A: ',A.shape) 
print('dimension B: ',B.shape)

#1) Permutar indices
Ap  = A.transpose(0,2,1,3)  #Aijmn 
Bp = B.transpose(0,3,1,2)   #Bmnkl

print('dimension Ap: ',Ap.shape)
print('dimension Bp: ',Bp.shape)

#2) Remodelar indices
App = Ap.reshape(i*j,m*n)   #A{ij}{mn}
Bpp = Bp.reshape(m*n,k*l)   #B{mn}{kl}

print('dimension App: ',App.shape)
print('dimension Bpp: ',Bpp.shape)

#3) Multiplicacion matricial
Cpp = np.matmul(App, Bpp)   #A{ij}{mn}B{mn}{kl}   

print('dimension Cpp: ',Cpp.shape)

#4) Reordenar tensor final
C   = Cpp.reshape(i,j,k,l)  #Cijkl

print('dimension C: ',C.shape)

dimension A:  (1, 2, 3, 4)
dimension B:  (2, 5, 6, 4)
dimension Ap:  (1, 3, 2, 4)
dimension Bp:  (2, 4, 5, 6)
dimension App:  (3, 8)
dimension Bpp:  (8, 30)
dimension Cpp:  (3, 30)
dimension C:  (1, 3, 5, 6)


In [9]:
#### Contraccion de tensor binario en tensorflow

i = 1; m = 2; j = 3; n = 4; k = 5; l = 6

A = net.add_node(np.random.rand(i,m,j,n))  
B = net.add_node(np.random.rand(m,k,l,n))

print('dimension A: ',A.tensor.shape)
print('dimension B: ',B.tensor.shape)

#Permutar indices

A.reorder_edges([A[0], A[2], A[1], A[3]]);  B.reorder_edges([B[0], B[3], B[1], B[2]])

# Conectar y remodelar indices

e1 = net.connect(A[2],B[0])
e2 = net.connect(A[3],B[1])

flattened_edge = net.flatten_edges([e1, e2])

print("dimension C: ",net.contract(flattened_edge).tensor.shape)


dimension A:  (1, 2, 3, 4)
dimension B:  (2, 5, 6, 4)
dimension C:  (1, 3, 5, 6)


# Costos de contraccion

Formas de calcular el costo de la contraccion tensorial:

![texto alternativo](https://static.wixstatic.com/media/d91e93_a03f6f99ee3f430ca094f6c7514870c3~mv2.png/v1/fill/w_948,h_201,al_c,q_90/D19.webp)

Hay dos formas para contraer una red con mas de dos tensores:



1.   En un solo paso como la suma directa de los indices internos de la red
2.   Una secuencia de N-1 contracciones binarias. 

![texto alternativo](https://static.wixstatic.com/media/d91e93_27f38171fdb04152a27551492a6c6b7d~mv2.png/v1/fill/w_750,h_83,al_c,q_90/d91e93_27f38171fdb04152a27551492a6c6b7d~mv2.webp)

![texto alternativo](https://static.wixstatic.com/media/d91e93_5ca5f98d307b404f8be574e105910826~mv2.png/v1/fill/w_750,h_250,al_c,q_90/d91e93_5ca5f98d307b404f8be574e105910826~mv2.webp)



In [10]:
##### Evaluar Red tensorial -A-B-C-

import time
start1 = time.time()

d = 20
A = np.random.rand(d,d) 
B = np.random.rand(d,d)
C = np.random.rand(d,d)
# Via sumatoria de indices internos
F0 = np.zeros((d,d))
for di in range(d):
    for dj in range(d):
        for dk in range(d):
            for dl in range(d):
                F0[di,dj] = F0[di,dj] + A[di,dk]*B[dk,dl]*C[dl,dj]
                
end1 = time.time()
print(end1 - start1)
            
start2 = time.time()
# Via secuencia de contracciones binarias
F1 = (A @ B) @ C 

end2 = time.time()
print(end2 - start2)

0.38547468185424805
0.0005705356597900391


# Contraccion de redes tensoriales

Formas de contraer una red tensorial eficientemente:

1.   Determinar la secuencia optima (Minimizar tamaño de tensores involucrados en la contraccion o ) de N-1 contracciones binarias
2.   Si la red es muy compleja puede usar algoritmos como el descrito en este articulo https://arxiv.org/abs/1304.6112

<img src="https://static.wixstatic.com/media/d91e93_0aec134314bc4a86b3fd10913c120f2f~mv2.png/v1/fill/w_644,h_525,al_c,q_90/d91e93_0aec134314bc4a86b3fd10913c120f2f~mv2.webp" width="400">



# ncon (Network contractor)

Disminuye el esfuerzo requerido para implementar una contracción. Esta función realiza automáticamente una secuencia deseada de permutaciones, remodelaciones y multiplicaciones de tensores necesarias para evaluar una red tensorial. El primer paso para usar 'ncon' para evaluar una red es hacer un diagrama etiquetado de la red de manera que:

*   Cada índice interno está etiquetado con un entero positivo único (típicamente enteros secuenciales que comienzan desde 1).

*   Los índices externos del diagrama (si hay alguno) están etiquetados con enteros negativos secuenciales [-1, -2, -3, ...] que denotan el orden de índice deseado en el tensor final (con -1 como primer índice, -2 como el segundo, etc.)

<img src="https://static.wixstatic.com/media/d91e93_eb675db6d96a47ce91a8ddbf265b2605~mv2.png/v1/fill/w_259,h_270,al_c,q_90/d91e93_eb675db6d96a47ce91a8ddbf265b2605~mv2.webp" width="150">

Después de esto, la rutina 'ncon' se llama de la siguiente manera,

**OutputTensor = ncon(TensorArray, IndexArray, con_order)**

In [11]:
from tensornetwork import ncon
sess = tf.InteractiveSession()

In [12]:
##### Contraccion usando ncon

#Ejemplo sencillo linea tensores -A-B-

a = tf.random_normal((2,2))
b = tf.random_normal((2,2))
c = ncon([a,b], [(-1,1),(1,-2)])
print(tf.norm(tf.matmul(a,b) - c).eval()) # imprime cero

0.0


In [13]:
#Ejemplo que se muestra en las figuras de arriba 

d = 10
A = tf.random_normal((d,d,d)); B = tf.random_normal((d,d,d,d))
C = tf.random_normal((d,d,d)); D = tf.random_normal((d,d))

#Tensores involucrados en la red
TensorArray = [A,B,C,D]

#etiquetado de indices
IndexArray = [[1,-2,2],[-1,1,3,4],[5,3,2],[4,5]]

#Uso de funcion ncon
E = ncon(TensorArray,IndexArray,con_order = [5,3,4,1,2])

print(E.shape)

(10, 10)
